In [1]:
import pandas as pd 
import numpy as np
from doepy import build
import seaborn as sns
import matplotlib.pyplot as plt
import operator
import random
import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
from IISEPaper import categorise
from IISEPaper import get_labeled_index
from IISEPaper import get_feature_label
from IISEPaper import get_labeled_set
from IISEPaper import get_training_Set
from IISEPaper import get_unlabeled_set
from IISEPaper import c_prediction
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from IISEPaper import predicted_region_divide
from IISEPaper import get_labeled_feasible_sample
from IISEPaper import get_feasible_rep
from IISEPaper import get_div_term
from IISEPaper import find_indices
from IISEPaper import get_select_index
from IISEPaper import GP_predict

In [5]:
from sklearn import preprocessing
s_scaler = preprocessing.StandardScaler()

#load the data you want here 
data = pd.read_csv('data_file2.csv')

ds,X_feature,y_ground,y_r = get_feature_label(data)
X_norm = s_scaler.fit_transform(X_feature)

In [6]:
data

,RampingRate,FlowRateGas,Time,PositionSulphur,PositionSilicon,LineWidth,Class
0,5,0.14,10,14,12,NaN,1
1,10,0.14,10,14,12,NaN,1
2,15,0.14,10,14,12,NaN,1
3,20,0.14,10,14,12,NaN,1
4,5,0.17,10,14,12,NaN,1
...,...,...,...,...,...,...,...
379,20,0.20,25,16,15,0.046793,0
380,5,0.23,25,16,15,0.048550,0
381,10,0.23,25,16,15,0.047449,0
382,15,0.23,25,16,15,0.047430,0


In [4]:
def remove_label(full_list,remove_list):
    for n in remove_list:
        while n in full_list:
            full_list.remove(n)
            
    return full_list

In [7]:
def get_index_div(div, unlabeled_index):
    index=[]
    max_div = max(div)
    temp = list(div).index(max_div)
    index.append(unlabeled_index[temp])
    return index

In [8]:
def get_div_norm(unlabeled_feature,labeled_feature):
    dis_div = euclidean_distances(unlabeled_feature,np.array(labeled_feature))
    div = dis_div.mean(axis=1)
    div = (div - div.min())/(div.max()-div.min())
    return div

In [9]:
def get_div_term(unlabeled_feature,labeled_feature):
    dis_div = euclidean_distances(unlabeled_feature,np.array(labeled_feature))
    div = dis_div.min(axis=1)
    #div = (div - div.min())/(div.max()-div.min())
    return div

In [10]:
def get_index_uncertainty(y_c_uncertainty, unlabeled_index,n):
    K = np.array(y_c_uncertainty)
    K.argsort()[-n:]
    selection_index = list(operator.itemgetter(*(list(K.argsort()[-n:])))(unlabeled_index))
    return selection_index

In [11]:
from sklearn.metrics.pairwise import euclidean_distances
def get_feasible_rep(c_pool,initial_feasible_sample):
    feasible_rep = euclidean_distances(c_pool,initial_feasible_sample)
    rep_new = feasible_rep.mean(axis=1)
    rep_new = 1- (rep_new - rep_new.min())/(rep_new.max()-rep_new.min())
    return rep_new

In [12]:
def get_rep_term(unlabeled_feature,labeled_feasible_sample):
    feasible_rep = euclidean_distances(unlabeled_feature,labeled_feasible_sample)
    rep_new = feasible_rep.mean(axis=1)
    rep_new = 1- (rep_new - rep_new.min())/(rep_new.max()-rep_new.min())
    return rep_new

In [13]:
def labeled_fea_sample(x_labeled,y_labeled,y_r_labeled):
    index_0 = np.where(y_labeled == 0)[0]
    index_1 = np.where(y_labeled == 1)[0]
    return x_labeled[index_0], x_labeled[index_1],y_r_labeled[index_0]

In [14]:
def predicted_fea_region(y_pred,unlabeled_feature):
    index_0 = np.where(y_pred == 0)[0]
    return unlabeled_feature[index_0]

In [15]:
def get_div_norm(unlabeled_feature,labeled_feature):
    dis_div = euclidean_distances(unlabeled_feature,np.array(labeled_feature))
    div = dis_div.mean(axis=1)
    div = (div - div.min())/(div.max()-div.min())
    return div

In [16]:
def find_rows(source, target):
    return np.where((source == target).all(axis=1))[0]

In [17]:
def get_rep_unlabeled(unlabeled_feature):
    A = euclidean_distances(unlabeled_feature,unlabeled_feature)
    m = A.shape[0]
    strided = np.lib.stride_tricks.as_strided
    s0,s1 = A.strides
    out = strided(A.ravel()[1:],shape=(m-1,m),strides =(s0+s1,s1)).reshape(m,-1)
    rep = out.mean(axis=1)
    rep_norm = 1- (rep - rep.min())/(rep.max()-rep.min())
    return rep_norm

In [18]:
def drop_na(x):
    return x[~np.isnan(x)]

In [19]:
model = GaussianProcessClassifier()

In [20]:
full_index = list(range(y_ground.shape[0]))

In [21]:
index_file = np.load('random_24_2d_index.npy',allow_pickle = True)

In [22]:
constraint_size = 2

In [23]:
acc=[]
result=[]
for n in range(len(index_file)):
    labeled_index= list(index_file[n])
    full_index = list(range(y_ground.shape[0]))

    x_labeled = X_norm[labeled_index]
    y_labeled = y_ground[labeled_index]
    y_r_labeled = y_r[labeled_index]
    unlabeled_index = remove_label(full_index,labeled_index)

    unlabeled_feature = X_norm[unlabeled_index]
    r_selected = y_r[labeled_index]

    f1=[]
    l=0
    r=[min(drop_na(r_selected))-0.03789]
    
    #if run the loop for objective optimization choose the stop criteria min(r) > 0 and len(r)<50
    #while min(r) > 0 and len(r) < 50:
    while l<20:
        
        #constraint model prediction
        print(x_labeled.shape)
        y_c_pred , y_prob , y_c_uncertainty = c_prediction(x_labeled, y_labeled, unlabeled_feature,model)
        y_c = y_ground.copy()
        y_c[unlabeled_index] = y_c_pred
        f1.append(accuracy_score(y_ground, y_c))

        #Set initial goast training set
        training_x = x_labeled
        training_y = r_selected.copy()
        labeled_feasible_sample, labeled_infeasible_sample,labeled_r = labeled_fea_sample(x_labeled,y_labeled,r_selected)
        y_pred_labeled_sudo,sigma_labeled_sudo = GP_predict(labeled_feasible_sample,labeled_r,labeled_infeasible_sample)
        training_y[np.where(y_labeled == 1)[0]]=y_pred_labeled_sudo


        #BO process
        # BO selection

        r_pred,r_sigma = GP_predict(training_x,training_y,unlabeled_feature)
        ucb = -r_pred + 2*r_sigma
        ucb = (1-y_c_pred) * (np.amax(y_prob, axis=1)) *ucb 
        ucb_max_index = np.argmax(ucb)

        ucb_selected_sample = unlabeled_feature[ucb_max_index]
        opt_sample_index = list(find_rows(X_norm,ucb_selected_sample))

        #update selected set 
        labeled_index.append(opt_sample_index[0])
       
        x_labeled = X_norm[labeled_index]
        unlabeled_index = remove_label(full_index,labeled_index)
      
        unlabeled_feature = X_norm[unlabeled_index]
        y_c_uncertainty = np.delete(y_c_uncertainty,ucb_max_index)
        
        n_ps = 0

        while n_ps<constraint_size:

            max_index_uncertainty = np.argmax(y_c_uncertainty)
            select_index_batch = unlabeled_index[max_index_uncertainty]
            labeled_index.append(select_index_batch) 
            x_labeled = X_norm[labeled_index]
            unlabeled_index = remove_label(full_index,labeled_index)
            unlabeled_feature = X_norm[unlabeled_index]
            y_c_uncertainty = np.delete(y_c_uncertainty,max_index_uncertainty)
         
            n_ps=n_ps+1

        # update selected samples with BO selection 

        y_labeled = y_ground[labeled_index]
        r_selected = y_r[labeled_index]

        y_r_labeled = y_r[labeled_index]
       
        r_set = y_r[labeled_index]
        
        r.append(min(drop_na(r_set))-0.03789)
       
        l=l+1
    result.append(r)
    acc.append(f1)
    print('Round {} finished'.format(n))

(24, 5)
(27, 5)
(30, 5)
(33, 5)
(36, 5)
(39, 5)
(42, 5)
(45, 5)
(48, 5)
(51, 5)
(54, 5)
(57, 5)
(60, 5)
(63, 5)
(66, 5)
(69, 5)
(72, 5)
(75, 5)
(78, 5)
(81, 5)
Round 0 finished
(24, 5)
(27, 5)
(30, 5)
(33, 5)
(36, 5)
(39, 5)
(42, 5)
(45, 5)
(48, 5)
(51, 5)
(54, 5)
(57, 5)
(60, 5)
(63, 5)
(66, 5)
(69, 5)
(72, 5)
(75, 5)
(78, 5)
(81, 5)
Round 1 finished
(24, 5)
(27, 5)
(30, 5)
(33, 5)
(36, 5)
(39, 5)
(42, 5)
(45, 5)
(48, 5)
(51, 5)
(54, 5)
(57, 5)
(60, 5)
(63, 5)
(66, 5)
(69, 5)
(72, 5)
(75, 5)
(78, 5)
(81, 5)
Round 2 finished
(24, 5)
(27, 5)
(30, 5)
(33, 5)
(36, 5)
(39, 5)
(42, 5)
(45, 5)
(48, 5)
(51, 5)
(54, 5)
(57, 5)
(60, 5)
(63, 5)
(66, 5)
(69, 5)
(72, 5)
(75, 5)
(78, 5)
(81, 5)
Round 3 finished
(24, 5)
(27, 5)
(30, 5)
(33, 5)
(36, 5)
(39, 5)
(42, 5)
(45, 5)
(48, 5)
(51, 5)
(54, 5)
(57, 5)
(60, 5)
(63, 5)
(66, 5)
(69, 5)
(72, 5)
(75, 5)
(78, 5)
(81, 5)
Round 4 finished
(24, 5)
(27, 5)
(30, 5)
(33, 5)
(36, 5)
(39, 5)
(42, 5)
(45, 5)
(48, 5)
(51, 5)
(54, 5)
(57, 5)
(60, 5)
(63, 5)
(66

In [24]:
mean_al=np.array(acc).mean(axis=0)
std_al = np.array(acc).std(axis=0)

In [25]:
mean_al

array([0.74440104, 0.76510417, 0.76992187, 0.77486979, 0.77955729,
       0.79453125, 0.80989583, 0.81848958, 0.82630208, 0.83619792,
       0.84375   , 0.84661458, 0.85416667, 0.85911458, 0.865625  ,
       0.86966146, 0.87734375, 0.88294271, 0.89075521, 0.89453125])

In [26]:
std_al

array([0.0461847 , 0.04247746, 0.03849655, 0.03699819, 0.03705131,
       0.03734869, 0.03587706, 0.03202278, 0.02933707, 0.03243102,
       0.03307394, 0.03543868, 0.03596202, 0.03205242, 0.02921313,
       0.02674157, 0.02481349, 0.0257467 , 0.02088982, 0.02019699])